## Notes:

Limiting scope: 
Kern County 
2010 - present

# Library Imports

In [2]:
import numpy as np
import pandas as pd

# Data Import checks

In [14]:
raw_drinkchem = pd.read_excel('./assets/Drinkingwater/chemical.xlsx')

In [15]:
print(raw_drinkchem.shape)
raw_drinkchem.head()

(1048575, 12)


,PRIM_STA_C,SAMP_DATE,SAMP_TIME,LAB_NUM,ANADATE,INDATE,METHOD,INBY,SPECIAL,STORE_NUM,XMOD,FINDING
0,0103039-004,2017-11-22,848,4790,2017-11-22,2017-11-30,SM 212,#,S,00081,<,3.0
1,0103039-004,2017-11-22,848,4790,2017-11-22,2017-11-30,SM 215,#,S,00086,<,1.0
2,0103039-004,2017-11-22,848,4790,2017-11-29,2017-11-30,SM 251,#,S,00095,NaN,550.0
3,0103039-004,2017-11-22,848,4790,2017-11-29,2017-11-30,SM4500,#,S,00403,NaN,8.1
4,0103039-004,2017-11-22,848,4790,2017-11-22,2017-11-30,300.0,#,S,00618,NaN,5.7


In [6]:
raw_drinkchem.columns

Index(['PRIM_STA_C', 'SAMP_DATE', 'SAMP_TIME', 'LAB_NUM', 'ANADATE', 'INDATE',
       'METHOD', 'INBY', 'SPECIAL', 'STORE_NUM', 'XMOD', 'FINDING'],
      dtype='object')

In [17]:
raw_drinkchem['PRIM_STA_C'].unique()

array(['0103039-004', '0103040-007', '0103041-001', ..., '1510005-031',
       '1510005-032', '1510005-033'], dtype=object)

In [24]:
kern_sample = [sample for sample in raw_drinkchem['PRIM_STA_C'] if sample.startswith('15')]

In [26]:
len(kern_sample)

196389

In [9]:
raw_drinkchem.describe().T

,count,mean,std,min,25%,50%,75%,max
SAMP_TIME,1000.0,1093.759000,255.380497,0.0,903.0,1237.0,1305.0000,1630.0
LAB_NUM,1000.0,2692.188000,1651.022176,10.0,1610.0,1675.0,2416.0000,9588.0
FINDING,1000.0,90.797538,416.041385,0.0,0.0,0.5,1.0775,7000.0


In [11]:
raw_drinkchem.describe(include='object').T

,count,unique,top,freq
PRIM_STA_C,1000,11,0104010-009,315
METHOD,253,29,E524.2,144
INBY,1000,2,#,536
SPECIAL,1000,1,S,1000
STORE_NUM,1000,206,01007,50
XMOD,543,2,<,540


In [9]:
raw_drinksys = pd.read_excel('./assets/Drinkingwater/watsys.xlsx')

In [13]:
print(raw_drinksys.shape)
raw_drinksys.head()

(1000, 11)


,SYSTEM_NO,SYSTEM_NAM,HQNAME,ADDRESS,CITY,STATE,ZIP,ZIP_EXT,POP_SERV,CONNECTION,AREA_SERVE
0,2800064,DAKOTA SHY WINERY,NaN,771 Sage Canyon Road,ST. HELENA,CA,94574,NaN,30,1,NaN
1,5403214,NaN,THREE RIVERS HISTORICAL MUSEUM,42268 SIERRA DR,THREE RIVERS,CA,93271,NaN,25,2,NaN
2,4000829,VINES R.V. RESORT,NaN,88 WELLSONA,PASO RO,CA,93446,NaN,400,1,NaN
3,103039,MOHRLAND MUTUAL WATER SYSTEM,MOHRLAND MUTUAL,24927 Mohr Drive,HAYWARD,CA,94545,tt,116,99,NaN
4,103040,NORRIS CANYON PROPERTY OWNERS ASSN.,NORRIS CANYON P,8653 Norris Canyon Road,CASTRO VALLEY,CA,94552,S,50,19,NaN


In [37]:
raw_drinksys['SYSTEM_NO'].unique()

array(['2800064', '5403214', '4000829', ..., '3107347', '1000644',
       '1504009'], dtype=object)

In [8]:
raw_drinksite = pd.read_excel('./assets/Drinkingwater/siteloc.xlsx')

In [9]:
print(raw_drinksite.shape)
raw_drinksite.head()

(58328, 11)


,PRI_STA_C,FRDS_NO,COUNTY,DISTRICT,USER_ID,SYSTEM_NO,WATER_TYPE,SOURCE_NAM,STATION_TY,STATUS,COMMENT_1
0,4301014-001,4301014001,43,17,SAN,4301014,G,WELL 01 - TREATED,NaN,AT,NaN
1,4301014-002,4301014002,43,17,SAN,4301014,G,WELL 01- CL2 TREATMENT,NaN,DT,NaN
2,2300906-002,2300906002,23,3,RXR,2300906,G,DBP SAMPLE LOCATION,NaN,DT,NaN
3,2310006-030,2310006030,23,3,RXR,2310006,G,DBP 781 TOKAY DRIVE,NaN,DT,NaN
4,2300955-001,2300955001,23,3,RXR,2300955,G,WELL 01,NaN,AR,NaN


In [7]:
raw_gw = pd.read_csv('./assets/Groundwater/KernEDF.txt', sep='\t')

In [8]:
raw_gw.head()

,COUNTY,GLOBAL_ID,FIELD_PT_NAME,LOGDATE,LOGTIME,LOGCODE,SAMPID,MATRIX,LABWO,LABCODE,...,ANADATE,BASIS,PARLABEL,PARVAL,PARVQ,LABDL,REPDL,UNITS,DILFAC,LNOTE
0,Kern,GAGW10000018,NCL-2-02,2014-11-21,930.0,BTSC,NCL-2-02,W,94042-1,TAIC,...,2014-11-25,N,BZ,0.000,ND,0.250,2.0,UG/L,1.0,NaN
1,Kern,GAGW10000018,NCL-2-02,2014-11-21,930.0,BTSC,NCL-2-02,W,94042-1,TAIC,...,2014-11-25,N,XYLO,0.000,ND,0.250,2.0,UG/L,1.0,NaN
2,Kern,GAGW10000018,NCL-2-02,2014-11-21,930.0,BTSC,NCL-2-02,W,94042-1,TAIC,...,2014-12-23,N,RA226228,0.691,=,0.377,5.0,PCI/L,1.0,NaN
3,Kern,GAGW10000018,NCL-28E4,2014-09-12,1000.0,BTSC,NCL-28E4,W,88029-1,TAIC,...,2014-09-13,N,HCO3,0.000,ND,4.800,4.8,MG/L,1.0,NaN
4,Kern,GAGW10000018,NCL-2-02,2016-05-10,908.0,BTST,Monitoring Well #2-02,W,NaN,BCLB,...,2016-05-17,N,CR,1.600,=,1.100,10.0,UG/L,1.0,"J,DX"


In [18]:
raw_gw.columns

Index(['COUNTY', 'GLOBAL_ID', 'FIELD_PT_NAME', 'LOGDATE', 'LOGTIME', 'LOGCODE',
       'SAMPID', 'MATRIX', 'LABWO ', 'LABCODE', 'LABSAMPID', 'ANMCODE',
       'LABLOTCTL', 'ANADATE', 'BASIS', 'PARLABEL', 'PARVAL', 'PARVQ', 'LABDL',
       'REPDL', 'UNITS', 'DILFAC', 'LNOTE'],
      dtype='object')

In [19]:
raw_gw['LOGDATE'].unique()

array(['2014-11-21', '2014-09-12', '2016-05-10', ..., '2006-06-30',
       '2013-06-12', nan], dtype=object)

In [36]:
raw_gw['FIELD_PT_NAME'].unique()

array(['NCL-2-02', 'NCL-28E4', 'QCTB', ..., 'A2-SW16', 'OW-21', nan],
      dtype=object)

In [27]:
raw_gw_loc = pd.read_excel('./assets/Groundwater/gama_location_kern_county.xlsx')

In [28]:
raw_gw_loc.shape

(1050, 5)

In [29]:
raw_gw_loc.head()

,DATASET,WELL ID,WELL NAME,LATITUDE,LONGITUDE
0,USGSNEW,USGS-345733118085201,USGS-345733118085201,34.959140,-118.148686
1,USGSNEW,USGS-351824117570101,USGS-351824117570101,35.306628,-117.951180
2,DHS,W0601502244,1502244-001,35.602611,-119.507496
3,DHS,W0601503431,1503431-003,35.715251,-119.427021
4,DHS,W0601510302,1510302-001,35.332059,-119.362728


In [35]:
raw_gw_loc['WELL NAME'].unique()

array(['USGS-345733118085201', 'USGS-351824117570101', '1502244-001', ...,
       '1510703-003', '1510703-001', '1502608-001'], dtype=object)